In [ ]:
from src.utils import *

# TODO: Document SQL query used to generate values in dates.txt
lists = sql_to_lists('../../data/dates.txt')
headers = csv[0]
rows = csv[1:]

'''
This is the technical, precise definition of "full participant", per
https://github.com/all-of-us/raw-data-repository/blob/08ef0e9ea9ee0ad2c7e19d6748371c775bb1d63a/rest-api/dao/participant_summary_dao.py#L39

 CASE WHEN (consent_for_study_enrollment = :submitted
                   AND consent_for_electronic_health_records = :submitted
                   AND num_completed_baseline_ppi_modules = :num_baseline_ppi_modules
                   AND physical_measurements_status = :completed
                   AND samples_to_isolate_dna = :received)
             THEN :full_participant
'''

# Count of participants who reached "full participant"
# status on a given day.
# 
# For example (not real data):
# {
#    'PITT': {
#       '2017-10-01': 50, 
#       '2017-10-02': 53,
#       ...
#    },
#    ...
# }
counts_by_site_per_day = {}

# for columns in rows[:10]: # For development
for row_columns in rows:
    # Each item in "rows" represents a participant -- particularly,
    # the set of dates when that participant completed a specified event
    # in the enrollment lifecycle
    hpo = row_columns[0]
    
    '''
    # Dead code, but perhaps useful documentation
    
    consent_for_study_enrollment_time = row_columns[2]
    consent_for_electronic_health_records_time = row_columns[3]
    physical_measurements_time = row_columns[4]
    
    # These correspond to "num_completed_baseline_ppi_modules"
    questionnaire_on_the_basics = row_columns[5]
    questionnaire_on_overall_health_time = row_columns[6]
    questionnaire_on_lifestyle_time = row_columns[7]
    
    # We assume these three samples comprise "samples_to_isolate_dna"
    sample_status_1ed04_time = row_columns[11] # EDTA DNA 4 mL 
    sample_status_1ed10_time = row_columns[12] # 1st EDTA DNA 10 mL 
    sample_status_2ed10_time = row_columns[13] # 2nd EDTA DNA 10 mL 
    '''
    
    # Gather all the dates of each lifecycle phase that needs to be 
    # passed in order to become a full participant
    dates = row_columns[2:8] + row_columns[11:14]
    
    # Get the latest -- the most recent -- of those dates
    most_recent_date = sorted(dates)[-1]
    
    # Increment by 1 the number of full participants enrolled 
    # in this HPO (i.e. high-level recruitment origin, "site") 
    # on this date
    if hpo in counts_by_site_per_day:
        counted_days = counts_by_site_per_day[hpo]
        if most_recent_date in counted_days:
            counts_by_site_per_day[hpo][most_recent_date] += 1
        else:
            counts_by_site_per_day[hpo][most_recent_date] = 1
    else:
        counts_by_site_per_day[hpo] = {most_recent_date: 1}
    
print(counts_by_site_per_day)
    